In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
def area(box):
    return (box[2] - box[0]) * (box[3] - box[1])


def intersection_over_union(boxes):
    assert(len(boxes) == 8)
    boxA = boxes[:4].values
    boxB = boxes[4:].values
    
    boxAArea = area(boxA)
    boxBArea = area(boxB)
    
    if (boxAArea == 0 or boxBArea == 0):
        return 0
        
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)

    
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

In [118]:
votes = pd.read_csv("train_data.csv")

In [119]:
votes_test=pd.read_csv("test_data.csv")

In [120]:
answers = pd.read_csv("train_answers.csv")

In [121]:
vote=votes[votes['itemId']!=31033]
votes=vote[vote['itemId']!=25135]

answer=answers[answers['itemId']!=31033]
answers=answer[answer['itemId']!=25135]

In [6]:
def ident(a):
    qw=(len(list(a))!=len(list(set(list(a)))))
    if qw==1:
        print a
    return qw
#list(votes_test.groupby('itemId')['userId'].apply(ident))

In [7]:
votes_test[votes_test['itemId']==146]

,userId,itemId,Xmin,Ymin,Xmax,Ymax
2782,1195,146,84,729,643,1091
2783,1383,146,120,735,602,1071
2784,422,146,45,757,634,1078
2785,1725,146,106,712,628,1118
3450,1195,146,51,226,542,545
3451,1383,146,83,231,506,528
3452,422,146,17,250,535,534
3453,1725,146,70,211,529,569


### Узнать количество юзеров голосовавших больше n раз

In [123]:
amount=votes.groupby('userId').apply(lambda x: x.shape[0])

In [122]:
b=list(set(list(votes['userId'])))
a=list(set(list(votes_test['userId'])))

In [124]:
iou_train=pd.Series()
for user in b:
    x=votes.loc[votes['userId']==user][['itemId','Xmin','Ymin', 'Xmax', 'Ymax']].reset_index().merge(answers, on='itemId')
    x["iou"] = x[['Xmin','Ymin', 'Xmax', 'Ymax', 'Xmin_true', 'Ymin_true', 'Xmax_true','Ymax_true']].apply(intersection_over_union, axis=1)
    r=x["iou"].mean()
    iou_train.at[user]=r

In [11]:
q1=list(amount[amount>20].keys())
q2=list(iou_train[iou_train>0.32].keys())
good_users=list(set(q1)&set(q2))
len(good_users)

24

## Обучаем

In [12]:
from sklearn.linear_model import LinearRegression as LR
from sklearn.linear_model import Ridge
rng = np.random.RandomState(0)
def apply_iou_score(x):
    if x in iou_train:
        return iou_train[x]
    else:
        return 0.445

In [28]:
vv=votes.copy()
vv['user_score']=votes['userId'].apply(lambda x: apply_iou_score(x))
votes=vv.copy()

In [13]:
def lin_reg_user(us_id):
    l={}
    x=votes.loc[votes['userId']==us_id]
    ss=list(set(x['itemId']))
    if len(ss)>0:
        ans_user=answers.loc[answers['itemId'].isin(ss)]
        for i in ['Xmin', 'Ymin', 'Xmax', 'Ymax']:
            l[i]=Ridge(alpha=3.0)
            igrek, iks=[],[]
            for ii in ss:
                igrek.append(list(ans_user.loc[ans_user['itemId']==ii][i+'_true'])[0])
                iks.append(list(x.loc[x['itemId']==ii][i])[0])
            l[i].fit(np.array(iks).reshape(-1,1), np.array(igrek))
        return l

In [14]:
u={}
for user in good_users:
    rrr=lin_reg_user(user)
    if rrr!=None:
        u[user]=rrr

In [15]:
len(u)


24

## Применяем на тестовой

In [158]:
def drop_under_thr(x, thr=0.32):
    scores=np.array(x['user_score'])
    len_scores=len(scores)
    scores_above_thr=[i for i in scores if i>thr]
    if len(scores_above_thr)>0:
        rez=x.loc[x['user_score']>thr][['userId','itemId','Xmin','Ymin', 'Xmax', 'Ymax','user_score']]
    else:
        rez=x.loc[x['user_score']==scores.max()][['userId','itemId','Xmin','Ymin', 'Xmax', 'Ymax','user_score']]
    return rez    

In [17]:
tt=[]
for user in a:
    if user in good_users:
        temp=[]
        for i in ['Xmin', 'Ymin', 'Xmax', 'Ymax']:
            temp.append(votes_test[votes_test['userId']==user][[i]].apply(lambda x: u[user][i].predict(np.array(x).reshape(-1,1))) )
        ite=votes_test[votes_test['userId']==user][['itemId','userId']]
        temp.append(ite)
        te=pd.concat(temp,axis=1)
        tt.append(te)
    else:
        #pass
        tt.append(votes_test[votes_test['userId']==user][[ 'Xmin', 'Ymin', 'Xmax', 'Ymax','itemId','userId']])
tt=pd.concat(tt) 
tess=tt[['itemId','userId','Xmin','Ymin', 'Xmax', 'Ymax']].copy()
tess['user_score']=tess['userId'].apply(lambda x: apply_iou_score(x))
test_votes_new=tess.groupby("itemId").apply(lambda x: drop_under_thr(x,0.32))

In [299]:
x_mi=test_votes_new[['userId','Xmin','Ymin', 'Xmax', 'Ymax','user_score']].groupby("itemId")[['Xmin']].quantile(.0)
x_ma=test_votes_new[['userId','Xmin','Ymin', 'Xmax', 'Ymax','user_score']].groupby("itemId")[['Xmax']].quantile(1.0)
y_mi=test_votes_new[['userId','Xmin','Ymin', 'Xmax', 'Ymax','user_score']].groupby("itemId")[['Ymin']].quantile(.0)
y_ma=test_votes_new[['userId','Xmin','Ymin', 'Xmax', 'Ymax','user_score']].groupby("itemId")[['Ymax']].quantile(1.0)
dd=x_mi.merge(y_mi,on=["itemId"])
dd=dd.merge(x_ma,on=["itemId"])
dd=dd.merge(y_ma,on=["itemId"]).reset_index()

dd['X_diap']= (-1.0)*(dd['Xmax']-dd['Xmin'])*0
dd['Y_diap']= (-1.0)*(dd['Ymax']-dd['Ymin'])*0
ddd=dd.copy()
ddd['Xmax']=dd['Xmax']-dd['X_diap']
ddd['Xmin']=dd['Xmin']+dd['X_diap']
ddd['Ymax']=dd['Ymax']
ddd['Ymin']=dd['Ymin']

ddd1=ddd[['itemId','Xmin','Ymin', 'Xmax', 'Ymax']]
ddd1.to_csv('submit_thr_goodusers.csv',index=0)

In [18]:
def viz(us):
    print u[us]['Xmax'].coef_[0],'x + ',u[us]['Xmax'].intercept_
    print u[us]['Xmin'].coef_[0],'x + ',u[us]['Xmin'].intercept_
    print u[us]['Ymax'].coef_[0],'x + ',u[us]['Ymax'].intercept_
    print u[us]['Ymin'].coef_[0],'x + ',u[us]['Ymin'].intercept_

## По соседям

In [125]:
#Генерируем юзеров с достаточным количеством точек и соседей
def gen_user_neighb(dots, neighb,thr=0):
    u=list(amount[amount>dots].keys())
    rez=[]
    for us in u:
        tt=True
        tt1=True
        l=list(votes[votes['userId']==us]['itemId'])
        for i in l:
            temp=list(votes[votes['itemId']==i]['userId'])
            temp.remove(us)
            if len(temp)==0: print us,i
            if len(temp)>=neighb:
                tt*=True
            else:
                tt*=False
            if ( False in map(lambda x: iou_train[x]>=thr,temp) ):
                tt1*=False
            else: tt1*=True
            
        if tt==True:
            if iou_train[us]>=thr:
                if tt1==True:
                    rez.append(us)
    return rez

In [201]:
yy=sorted(gen_user_neighb(9,4,0.0))
print len(yy)

4


In [202]:
r=[]
for us in yy:
    l=list(votes[votes['userId']==us]['itemId'])
    for i in l:
        temp=list(votes[votes['itemId']==i]['userId'])
        temp.remove(us)
        no_user=votes.loc[votes['itemId']==i].loc[votes['userId'].isin(temp)][['Xmin','Ymin', 'Xmax', 'Ymax']].apply(np.mean)
        www=pd.DataFrame(no_user).transpose()
        www['userId']=us
        www['itemId']=i
        r.append(www)
rez=pd.concat(r)

In [132]:
from sklearn.linear_model import Lasso, ElasticNet, Ridge, RidgeCV, LassoCV
from sklearn.linear_model import LinearRegression as LR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [203]:
def lin_reg(us_id):
    x=rez.loc[rez['userId']==us_id]
    xx=votes.loc[votes['userId']==us_id]
    ss=list(x['itemId'])
    ans_user=answers.loc[answers['itemId'].isin(ss)]    
    l={}
    for i in ['Xmin', 'Ymin', 'Xmax', 'Ymax']:
        l[i]=LR()
        igrek, iks=[],[]
        for ii in ss:
            igrek.append(list(ans_user.loc[ans_user['itemId']==ii][i+'_true'])[0])
            iks.append([list(xx.loc[xx['itemId']==ii][i])[0], list(x.loc[x['itemId']==ii][i])[0] ])
        l[i].fit(np.array(iks), np.array(igrek))
    return l

def rf_reg(us_id,nn):
    x=rez.loc[rez['userId']==us_id]
    xx=votes.loc[votes['userId']==us_id]
    ss=list(x['itemId'])
    ans_user=answers.loc[answers['itemId'].isin(ss)]    
    l={}
    for i in ['Xmin', 'Ymin', 'Xmax', 'Ymax']:
        l[i]=RandomForestRegressor(n_estimators=nn)
        igrek, iks=[],[]
        for ii in ss:
            igrek.append(list(ans_user.loc[ans_user['itemId']==ii][i+'_true'])[0])
            iks.append([list(xx.loc[xx['itemId']==ii][i])[0], list(x.loc[x['itemId']==ii][i])[0] ])
        l[i].fit(np.array(iks), np.array(igrek))
    return l

def lasso_reg(us_id,alp):
    x=rez.loc[rez['userId']==us_id]
    xx=votes.loc[votes['userId']==us_id]
    ss=list(x['itemId'])
    ans_user=answers.loc[answers['itemId'].isin(ss)]    
    l={}
    for i in ['Xmin', 'Ymin', 'Xmax', 'Ymax']:
        l[i]=Lasso(alpha=alp)
        igrek, iks=[],[]
        for ii in ss:
            igrek.append(list(ans_user.loc[ans_user['itemId']==ii][i+'_true'])[0])
            iks.append([list(xx.loc[xx['itemId']==ii][i])[0], list(x.loc[x['itemId']==ii][i])[0] ])
        l[i].fit(np.array(iks), np.array(igrek))
    return l

def ridge_reg(us_id,alp):
    x=rez.loc[rez['userId']==us_id]
    xx=votes.loc[votes['userId']==us_id]
    ss=list(x['itemId'])
    ans_user=answers.loc[answers['itemId'].isin(ss)]    
    l={}
    for i in ['Xmin', 'Ymin', 'Xmax', 'Ymax']:
        l[i]=Ridge(alpha=alp)
        igrek, iks=[],[]
        for ii in ss:
            igrek.append(list(ans_user.loc[ans_user['itemId']==ii][i+'_true'])[0])
            iks.append([list(xx.loc[xx['itemId']==ii][i])[0], list(x.loc[x['itemId']==ii][i])[0] ])
        l[i].fit(np.array(iks), np.array(igrek))
    return l

def ridgecv_reg(us_id):
    x=rez.loc[rez['userId']==us_id]
    xx=votes.loc[votes['userId']==us_id]
    ss=list(x['itemId'])
    ans_user=answers.loc[answers['itemId'].isin(ss)]    
    l={}
    for i in ['Xmin', 'Ymin', 'Xmax', 'Ymax']:
        l[i]=RidgeCV(alphas=np.arange(1, 1000000, 10))
        igrek, iks=[],[]
        for ii in ss:
            igrek.append(list(ans_user.loc[ans_user['itemId']==ii][i+'_true'])[0])
            iks.append([list(xx.loc[xx['itemId']==ii][i])[0], list(x.loc[x['itemId']==ii][i])[0] ])
        l[i].fit(np.array(iks), np.array(igrek))
    return l

def gbm_reg(us_id,lr,nn):
    x=rez.loc[rez['userId']==us_id]
    xx=votes.loc[votes['userId']==us_id]
    ss=list(x['itemId'])
    ans_user=answers.loc[answers['itemId'].isin(ss)]    
    l={}
    for i in ['Xmin', 'Ymin', 'Xmax', 'Ymax']:
        l[i]=GradientBoostingRegressor(learning_rate=lr, n_estimators=nn,max_depth=1)
        igrek, iks=[],[]
        for ii in ss:
            igrek.append(list(ans_user.loc[ans_user['itemId']==ii][i+'_true'])[0])
            iks.append([list(xx.loc[xx['itemId']==ii][i])[0], list(x.loc[x['itemId']==ii][i])[0] ])
        l[i].fit(np.array(iks), np.array(igrek))
    return l

def en_reg(us_id,alp,l):
    x=rez.loc[rez['userId']==us_id]
    xx=votes.loc[votes['userId']==us_id]
    ss=list(x['itemId'])
    ans_user=answers.loc[answers['itemId'].isin(ss)]    
    l={}
    for i in ['Xmin', 'Ymin', 'Xmax', 'Ymax']:
        l[i]=ElasticNet(alpha=alp,l1_ratio=l)
        igrek, iks=[],[]
        for ii in ss:
            igrek.append(list(ans_user.loc[ans_user['itemId']==ii][i+'_true'])[0])
            iks.append([list(xx.loc[xx['itemId']==ii][i])[0], list(x.loc[x['itemId']==ii][i])[0] ])
        l[i].fit(np.array(iks), np.array(igrek))
    return l

In [204]:
RIDGECV_Reg={}
for user in yy:
    print user
    RIDGECV_Reg[user]=ridgecv_reg(user)

25
242
651
761


In [205]:
for us in yy:
    for i in ['Xmin', 'Ymin', 'Xmax', 'Ymax']:
        print us,i,RIDGECV_Reg[us][i].alpha_

25 Xmin 3851
25 Ymin 1981
25 Xmax 4091
25 Ymax 21371
242 Xmin 33631
242 Ymin 147911
242 Xmax 169761
242 Ymax 18011
651 Xmin 657771
651 Ymin 51221
651 Xmax 202251
651 Ymax 104701
761 Xmin 35951
761 Ymin 1801
761 Xmax 30021
761 Ymax 2221


In [206]:
'''
Lin_Reg={}
for user in yy:
    Lin_Reg[user]=lin_reg(user)

Enet_2={}
for user in yy:
    Enet_2[user]=en_reg(user,2,1)

Lasso_2={}
for user in yy:
    Lasso_2[user]=lasso_reg(user,2)
    
RF_Reg10={}
for user in yy:
    RF_Reg10[user]=rf_reg(user,10)

RF_Reg20={}
for user in yy:
    RF_Reg20[user]=rf_reg(user,20)
    
RF_Reg30={}
for user in yy:
    RF_Reg30[user]=rf_reg(user,30)
    
RIDGE_Reg1={}
for user in yy:
    RIDGE_Reg1[user]=ridge_reg(user,1)
    
RIDGE_Reg4={}
for user in yy:
    RIDGE_Reg4[user]=ridge_reg(user,4)

GBM_Reg_01_50={}
for user in yy:
    GBM_Reg_01_50[user]=gbm_reg(user,0.1,50)
    
GBM_Reg_004_40={}
for user in yy:
    GBM_Reg_004_40[user]=gbm_reg(user,0.04,40)
'''
GBM_Reg_01_20={}
for user in yy:
    GBM_Reg_01_20[user]=gbm_reg(user,0.1,20)

In [207]:
tt=[]
for user in a:
    if user in yy:
        r=[]
        l=list(votes_test[votes_test['userId']==user]['itemId'])
        for i in l:
            temp=list(votes_test[votes_test['itemId']==i]['userId'])
            temp.remove(user)
            if len(temp)==0:
                print 'Len = 0', user, i
                tt.append(votes_test[votes_test['userId']==user][[ 'Xmin', 'Ymin', 'Xmax', 'Ymax','itemId','userId']])
            else:
                no_user=votes_test.loc[votes_test['itemId']==i].loc[votes_test['userId'].isin(temp)][['Xmin','Ymin', 'Xmax', 'Ymax']].apply(np.mean)
                www=pd.DataFrame(no_user).transpose()
                www['userId']=us
                www['itemId']=i
                a1=votes_test.loc[votes_test['itemId']==i].loc[votes_test['userId']==user]
                otvet=pd.DataFrame()
                index=a1['userId'].keys()[0]
                otvet.at[index,'userId']=int(user)
                otvet.at[index,'itemId']=i
                for i in ['Xmin', 'Ymin', 'Xmax', 'Ymax']:
                    temp1=np.array([list(a1[i])[0], list(www[i])[0]])
                    '''
                    p1=Lin_Reg[user][i].predict( temp1.reshape(1,-1) )[0]
                    p2=RF_Reg10[user][i].predict( temp1.reshape(1,-1) )[0]
                    p3=RF_Reg20[user][i].predict( temp1.reshape(1,-1) )[0]
                    p4=RIDGE_Reg1[user][i].predict( temp1.reshape(1,-1) )[0]
                    p5=RIDGE_Reg4[user][i].predict( temp1.reshape(1,-1) )[0]
                    p6=GBM_Reg_01_50[user][i].predict( temp1.reshape(1,-1) )[0]
                    p7=GBM_Reg_005_100[user][i].predict( temp1.reshape(1,-1) )[0]
                    p8=GBM_Reg_004_40[user][i].predict( temp1.reshape(1,-1) )[0]
                    p9=Lasso_2[user][i].predict( temp1.reshape(1,-1) )[0]
                    p10=RF_Reg30[user][i].predict( temp1.reshape(1,-1) )[0]
                    
                    '''
                    p11=GBM_Reg_01_20[user][i].predict( temp1.reshape(1,-1) )[0]
                    p12=RIDGECV_Reg[user][i].predict( temp1.reshape(1,-1) )[0]
                    ppp=np.array([p11,p12]).mean()
                    otvet.at[index,i]=ppp
                    
                    
                    
                    
                tt.append(otvet)
    else:
        tt.append(votes_test[votes_test['userId']==int(user)][[ 'Xmin', 'Ymin', 'Xmax', 'Ymax','itemId','userId']])
tt=pd.concat(tt) 
tess=tt[['itemId','userId','Xmin','Ymin', 'Xmax', 'Ymax']].copy()
tess['user_score']=tess['userId'].apply(lambda x: apply_iou_score(x))
test_votes_new=tess.groupby("itemId").apply(lambda x: drop_under_thr(x,0.0))


/home/jerry/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:48: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [208]:
x_mi=test_votes_new[['userId','Xmin','Ymin', 'Xmax', 'Ymax','user_score']].groupby("itemId")[['Xmin']].quantile(.0)
x_ma=test_votes_new[['userId','Xmin','Ymin', 'Xmax', 'Ymax','user_score']].groupby("itemId")[['Xmax']].quantile(1.0)
y_mi=test_votes_new[['userId','Xmin','Ymin', 'Xmax', 'Ymax','user_score']].groupby("itemId")[['Ymin']].quantile(.0)
y_ma=test_votes_new[['userId','Xmin','Ymin', 'Xmax', 'Ymax','user_score']].groupby("itemId")[['Ymax']].quantile(1.0)
dd=x_mi.merge(y_mi,on=["itemId"])
dd=dd.merge(x_ma,on=["itemId"])
dd=dd.merge(y_ma,on=["itemId"]).reset_index()

dd['X_diap']= (-1.0)*(dd['Xmax']-dd['Xmin'])*0
dd['Y_diap']= (-1.0)*(dd['Ymax']-dd['Ymin'])*0
ddd=dd.copy()
ddd['Xmax']=dd['Xmax']-dd['X_diap']
ddd['Xmin']=dd['Xmin']+dd['X_diap']
ddd['Ymax']=dd['Ymax']
ddd['Ymin']=dd['Ymin']

ddd1=ddd[['Xmin','Ymin', 'Xmax', 'Ymax']].copy()
ddd1['itemId']=ddd['itemId'].apply(lambda x: np.int(x))
ddd2=ddd1[['itemId','Xmin','Ymin', 'Xmax', 'Ymax']]
ddd2.to_csv('submit_thr_sosedi.csv',index=0)